In [44]:
%load_ext autoreload
%autoreload 2

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import classification_report

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import pandas as pd
from DataGenerator import DataGenerator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
X_train = np.load('./datasets/CNN/X_train_no_edge_frames.npy')
X_test = np.load('./datasets/CNN/X_test_no_edge_frames.npy')
y_train = np.load('./datasets/CNN/y_train_no_edge_frames.npy')
y_test = np.load('./datasets/CNN/y_test_no_edge_frames.npy')

In [45]:
def createModel():
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=(3,3), input_shape=(176,240,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3)))
    
    model.add(Conv2D(filters=32, kernel_size=(3,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3)))
    
    model.add(Conv2D(filters=64, kernel_size=(3,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3)))
    
    model.add(GlobalAveragePooling2D())
    
#     model.add(Dense(64, activation='relu', input_dim=2048))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(500, activation='relu', input_dim=2048))
    model.add(Flatten())
    model.add(Dense(48, activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [46]:
model = createModel()

In [47]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 174, 238, 16)      160       
_________________________________________________________________
batch_normalization_7 (Batch (None, 174, 238, 16)      64        
_________________________________________________________________
activation_7 (Activation)    (None, 174, 238, 16)      0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 58, 79, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 56, 77, 32)        4640      
_________________________________________________________________
batch_normalization_8 (Batch (None, 56, 77, 32)        128       
_________________________________________________________________
activation_8 (Activation)    (None, 56, 77, 32)       

In [48]:
train_gen = DataGenerator(X_train, y_train, 128)
test_gen = DataGenerator(X_test, y_test, 128)

In [49]:
model.fit_generator(generator=train_gen,
                    steps_per_epoch=len(train_gen),
                    #                    steps_per_epoch = int(3800 // batch_size),
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_gen,
                    validation_steps=len(test_gen),
                    #                    validation_steps = int(950 // batch_size)
                   )

Epoch 1/10
283/283 [==============================] - 70s 248ms/step - loss: 3.5665 - accuracy: 0.0853 - val_loss: 3.7199 - val_accuracy: 0.0876
Epoch 2/10
283/283 [==============================] - 68s 240ms/step - loss: 3.5100 - accuracy: 0.0911 - val_loss: 3.5481 - val_accuracy: 0.0920
Epoch 3/10
283/283 [==============================] - 68s 239ms/step - loss: 3.5035 - accuracy: 0.0935 - val_loss: 3.5018 - val_accuracy: 0.0910
Epoch 4/10
283/283 [==============================] - 66s 234ms/step - loss: 3.4954 - accuracy: 0.0962 - val_loss: 3.5351 - val_accuracy: 0.0833
Epoch 5/10
283/283 [==============================] - 66s 234ms/step - loss: 3.4864 - accuracy: 0.0956 - val_loss: 3.5266 - val_accuracy: 0.0793
Epoch 6/10
283/283 [==============================] - 66s 234ms/step - loss: 3.4780 - accuracy: 0.1054 - val_loss: 3.5452 - val_accuracy: 0.0772
Epoch 7/10
283/283 [==============================] - 67s 237ms/step - loss: 3.4705 - accuracy: 0.1080 - val_loss: 3.4895 - val_ac

In [52]:
history = _49

In [ ]:
estimator = KerasClassifier(build_fn=createModel, epochs=400, batch_size=128)

In [106]:
kfold = KFold(n_splits=5, shuffle=True)

In [107]:
early_stop = EarlyStopping(patience=10, verbose=0)

In [108]:
val_scores = [] 

In [109]:
for i, (train_ix, val_ix) in enumerate(kfold.split(X_train, y_train)):
    model = createModel()
    model.fit(X_train[train_ix], y_train[train_ix], epochs=400,
              callbacks=[early_stop], batch_size=128,
              validation_data=(X_train[val_ix], y_train[val_ix]))
    val_scores.append(model.evaluate(X_train[val_ix], y_train[val_ix], batch_size=128))

Train on 30364 samples, validate on 7592 samples
Epoch 1/400
30364/30364 [==============================] - 7s 245us/step - loss: 3.4765 - accuracy: 0.1059 - val_loss: 3.4069 - val_accuracy: 0.1258
Epoch 2/400
30364/30364 [==============================] - 7s 238us/step - loss: 3.3511 - accuracy: 0.1285 - val_loss: 3.3593 - val_accuracy: 0.1254
Epoch 3/400
30364/30364 [==============================] - 7s 225us/step - loss: 3.2810 - accuracy: 0.1426 - val_loss: 3.3032 - val_accuracy: 0.1382
Epoch 4/400
30364/30364 [==============================] - 7s 233us/step - loss: 3.2169 - accuracy: 0.1540 - val_loss: 3.2999 - val_accuracy: 0.1322
Epoch 5/400
30364/30364 [==============================] - 7s 227us/step - loss: 3.1718 - accuracy: 0.1605 - val_loss: 3.2583 - val_accuracy: 0.1486
Epoch 6/400
30364/30364 [==============================] - 7s 240us/step - loss: 3.1233 - accuracy: 0.1712 - val_loss: 3.2265 - val_accuracy: 0.1520
Epoch 7/400
30364/30364 [==============================] 

KeyboardInterrupt: 

In [97]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [0.1, 1]},
                    {'kernel': ['linear'], 'C': [0.1, 1]}]
scores = ['precision_macro', 'recall_macro', 'f1_macro']

In [98]:
clf = GridSearchCV(SVC(), tuned_parameters, scoring=scores, refit='f1_macro', verbose=10, n_jobs=-1)
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print(clf.best_params_)

print("Scoring: ")
print(clf.score(X_test, y_test))


clf.best_estimator_

/home/fabiana/miniconda3/envs/opencv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   40.4s
[Parallel(n_jobs=-1)]: Done  13 out of  18 | elapsed:   51.9s remaining:   20.0s
[Parallel(n_jobs=-1)]: Done  15 out of  18 | elapsed:   53.6s remaining:   10.7s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.1min finished
/home/fabiana/miniconda3/envs/opencv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best parameters set found on development set:
{'C': 0.1, 'kernel': 'linear'}
Scoring: 
0.04748322972335826


/home/fabiana/miniconda3/envs/opencv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/fabiana/miniconda3/envs/opencv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)